In [2]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from scipy.signal import convolve
import xarray as xr
from scipy.special import softmax
import pygmt
import plotly.express as px
from smount_predictors import SeamountHelp
import pandas as pd
data_dir = Path('data') / 'vggmask' / 'vgg_8km'
pd.options.plotting.backend = "plotly"

In [3]:
training_bounds = SeamountHelp.readKMLbounds(Path('data') / 'Seamount_training_zone.kml')
training_data = SeamountHelp.readAndFilterGRD(Path('data') / 'vgg_swot.grd', training_bounds[:2], training_bounds[2:])

In [4]:
training_data = SeamountHelp.adjust_lon(training_data)

In [5]:
convolutions = {}
for file in data_dir.glob('VGG_*_3*.grd'):
    kernel = SeamountHelp.readAndFilterGRD(file, (-0.2, 0.2), (-130.1, -129.9))
    kernel = SeamountHelp.normalize_kernel(kernel)['z']
    to_modify = training_data.copy()
    to_modify['z'].values = convolve(training_data['z'].values, kernel.values, mode='same')
    
    # to_modify = SeamountHelp.inverse_adjust_lon(to_modify)
    convolutions[file.stem] = to_modify

In [6]:
convolution_plots = SeamountHelp.show_convolutions(convolutions)

In [7]:
max_vals = {v['z'].values.max(): k for k, v in convolutions.items()}

In [8]:
max_vals[max(max_vals.keys())]

'VGG_100_3000'

In [11]:
SeamountHelp.plot_xarr(convolutions["VGG_100_3000"], name="VGG_100_3000").update_coloraxes(colorscale='Viridis')